<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/11-deep-learning-for-text/04_machine_translation_sequence_to_sequence_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Machine translation: A sequence-to-sequence learning

In this notebook, you’ll deepen your expertise by learning about
sequence-to-sequence models.

A sequence-to-sequence model takes a sequence as input (often a sentence or
paragraph) and translates it into a different sequence. This is the task at the heart of many of the most successful applications of NLP:
- **Machine translation**—Convert a paragraph in a source language to its equivalent in a target language.
- **Text summarization**—Convert a long document to a shorter version that retains the most important information.
- **Question answering**—Convert an input question into its answer.
- **Chatbots**—Convert a dialogue prompt into a reply to this prompt, or convert the history of a conversation into the next reply in the conversation.
- **Text generation**—Convert a text prompt into a paragraph that completes the prompt.

The general template behind sequence-to-sequence models is described in figure.

<img src='https://github.com/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/11-deep-learning-for-text/images/3.png?raw=1' width='800'/>

During training:-
- An `encoder` model turns the source sequence into an intermediate representation.
- A `decoder` is trained to predict the next token i in the target sequence by looking at both previous tokens `(0 to i - 1)` and the encoded source sequence.

**During inference, we don’t have access to the target sequence**—we’re trying to predict it from scratch. We’ll have to generate it one token at a time:

- We obtain the encoded source sequence from the encoder.
- The decoder starts by looking at the encoded source sequence as well as an initial “seed” token (such as the string `[start]`), and uses them to predict the first real token in the sequence.
- The predicted sequence so far is fed back into the decoder, which generates the next token, and so on, until it generates a stop token (such as the string
`[end]`).

Everything you’ve learned so far can be repurposed to build this new kind of model.

Let’s dive in.


##Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import random
import string
import re

import numpy as np

We’ll be working with an English-to-Spanish translation dataset available at
www.manythings.org/anki/. 

Let’s download it:

In [ ]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

##Data preparation

The text file contains one example per line: an English sentence, followed by a tab character, followed by the corresponding Spanish sentence. 

Let’s parse this file.

In [ ]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
  lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
  # Each line contains an English phrase and its Spanish translation, tab-separated.
  english, spanish = line.split("\t")
  # We prepend "[start]" and append "[end]" to the Spanish sentence, to match the template
  spanish = "[start]" + spanish + "[end]"
  text_pairs.append((english, spanish))

Our `text_pairs` look like this:

In [ ]:
print(random.choice(text_pairs))

('I helped my mother with the cooking.', '[start]Ayudé a cocinar a mi mamá.[end]')


In [ ]:
print(random.choice(text_pairs))

('Tom saw it.', '[start]Tom lo vio.[end]')


Let’s shuffle them and split them into the usual training, validation, and test sets:

In [ ]:
random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples

train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples: num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

Next, let’s prepare two separate TextVectorization layers: one for English and one for Spanish. 

We’re going to need to customize the way strings are preprocessed:

In [ ]:
# Prepare a custom string standardization function for the Spanish TextVectorization layer: it preserves [ and ] 
# but strips ¿ (as well as all other characters from strings.punctuation).
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# To keep things simple, we’ll only look at the top 15,000 words in each language, and we’ll restrict sentences to 20 words.
vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length)
# Generate Spanish sentences that have one extra token, since we’ll need to offset the sentence by one step during training.
target_vectorization = layers.TextVectorization(max_tokens=vocab_size, output_mode="int", 
                                                output_sequence_length=sequence_length + 1,
                                                standardize=custom_standardization)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
# Learn the vocabulary of each language
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

Finally, we can turn our data into a tf.data pipeline. 

We want it to return a tuple `(inputs, target)` where `inputs` is a dict with two keys,`encoder_inputs` (the English sentence) and `decoder_inputs` (the Spanish sentence), and `target` is the Spanish sentence offset by one step ahead.

In [ ]:
batch_size = 64

def format_dataset(eng, spa):
  eng = source_vectorization(eng)
  spa = target_vectorization(spa)
  return (
      {
      "english": eng,
      "spanish": spa[:, :-1],  # The input Spanish sentence doesn’t include the last token to keep inputs and targets at the same length
      },
      spa[:, 1:]   # The target Spanish sentence is one step ahead. Both are still the same length (20 words)
  )

In [ ]:
def make_dataset(pairs):
  eng_texts, spa_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  spa_texts = list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls=4)
  # Use in-memory caching to speed up preprocessing
  return dataset.shuffle(2048).prefetch(16).cache()

In [ ]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Here’s what our dataset outputs look like:

In [ ]:
for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape: {inputs['english'].shape}")
  print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
  print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


The data is now ready—time to build some models. We’ll start with a recurrent
sequence-to-sequence model before moving on to a Transformer.

##Sequence-to-sequence learning with RNNs